<a href="https://colab.research.google.com/github/jyoti612/English-Hindi-translator-using-parallel-corporus/blob/main/FINAL_YEAR_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,TimeDistributed,Embedding,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from string import digits
import nltk
import re
import string
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  app.launch_new_instance()


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
filename = '/content/drive/My Drive/ML DATASETS/hindi_english_parallel.csv'
lines= pd.read_csv(filename)

Mounted at /content/drive


In [ ]:
lines = lines[:30000]
lines.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है,A list of plugins that are disabled by default


In [ ]:
lines['english']=lines['english'].apply(lambda x: str(x))
lines['hindi']=lines['hindi'].apply(lambda x: str(x))
lines['english']=lines['english'].apply(lambda x: x.lower())
lines['hindi']=lines['hindi'].apply(lambda x: x.lower())

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

lines['english']=lines['english'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
lines['hindi']=lines['hindi'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
lines['hindi'][0]

'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'

In [ ]:
lines['english']=lines['english'].apply(lambda x: re.sub("'", '', x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub("'", '', x))

In [ ]:
lines.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,give application accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,accerciser accessibility explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,default plugin layout bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,default plugin layout top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है,list plugins disabled default


In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi']=lines['hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
lines.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,give application accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,accerciser accessibility explorer
2,निचले पटल के लिए डिफोल्ट प्लगइन खाका,default plugin layout bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,default plugin layout top panel
4,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है,list plugins disabled default


In [ ]:
remove_digits = str.maketrans('', '', digits)

In [ ]:
remove_digits

{48: None,
 49: None,
 50: None,
 51: None,
 52: None,
 53: None,
 54: None,
 55: None,
 56: None,
 57: None}

In [ ]:
a = lines['english'][0].translate(remove_digits)

In [ ]:
a

'give application accessibility workout'

In [ ]:
a.strip()

'give application accessibility workout'

In [ ]:
remove_digits = str.maketrans('', '', digits)
lines['english']=lines['english'].apply(lambda x: x.translate(remove_digits))
lines['hindi']=lines['hindi'].apply(lambda x: x.translate(remove_digits))

lines['hindi'] = lines['hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english']=lines['english'].apply(lambda x: x.strip())
lines['hindi']=lines['hindi'].apply(lambda x: x.strip())
lines['english']=lines['english'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub(" +", " ", x))

In [ ]:
'hello! how are you buddy?'.strip()

'hello! how are you buddy?'

In [ ]:
lines['english'][0]

'give application accessibility workout'

In [ ]:
lines['hindi'] = lines['hindi'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines['hindi'][0]

'START_ अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें _END'

In [ ]:
all_eng_words=set()
for eng in lines['english']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [ ]:
lines.head()

,hindi,english
0,START_ अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें _END,give application accessibility workout
1,START_ एक्सेर्साइसर पहुंचनीयता अन्वेषक _END,accerciser accessibility explorer
2,START_ निचले पटल के लिए डिफोल्ट प्लगइन खाका _END,default plugin layout bottom panel
3,START_ ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका _END,default plugin layout top panel
4,START_ उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है _END,list plugins disabled default


In [ ]:
lines['length_eng']=lines['english'].apply(lambda x:len(x.split(" ")))
lines['length_hin']=lines['hindi'].apply(lambda x:len(x.split(" ")))
lines.head()
lines[lines['length_eng']>30].shape

(10, 4)

In [ ]:
lines=lines[lines['length_eng']<=20]
lines=lines[lines['length_hin']<=20]
print("maximum length of Hindi Sentence ",max(lines['length_hin']))
print("maximum length of English Sentence ",max(lines['length_eng']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  19


In [ ]:
max_length_src=max(lines['length_hin'])
max_length_tar=max(lines['length_eng'])
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(2620, 3407)

In [ ]:
num_decoder_tokens

3407

In [ ]:
num_decoder_tokens += 1

In [ ]:
num_decoder_tokens


3408

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
input_token_index

{'a': 1,
 'able': 2,
 'abort': 3,
 'aborted': 4,
 'above': 5,
 'absolute': 6,
 'ac': 7,
 'accelerator': 8,
 'accelerator…': 9,
 'accentuated': 10,
 'accept': 11,
 'acceptable': 12,
 'accepted': 13,
 'accerciser': 14,
 'access': 15,
 'accessed': 16,
 'accessibility': 17,
 'accessible': 18,
 'accessibles': 19,
 'according': 20,
 'accordion': 21,
 'ace': 22,
 'aces': 23,
 'across': 24,
 'act': 25,
 'acti': 26,
 'action': 27,
 'actionable': 28,
 'actions': 29,
 'activatable': 30,
 'activate': 31,
 'activated': 32,
 'active': 33,
 'actual': 34,
 'adaptive': 35,
 'add': 36,
 'added': 37,
 'addition': 38,
 'additional': 39,
 'addrcheck': 40,
 'address': 41,
 'addressbook': 42,
 'addresses': 43,
 'adds': 44,
 'add…': 45,
 'advance': 46,
 'advanced': 47,
 'advised': 48,
 'affects': 49,
 'again': 50,
 'agnes': 51,
 'aim': 52,
 'aisleriot': 53,
 'al': 54,
 'albert': 55,
 'alert': 56,
 'algorithm': 57,
 'alidate': 58,
 'all': 59,
 'alleys': 60,
 'allocate': 61,
 'allocated': 62,
 'allow': 63,
 'al

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
reverse_input_char_index

{1: 'a',
 2: 'able',
 3: 'abort',
 4: 'aborted',
 5: 'above',
 6: 'absolute',
 7: 'ac',
 8: 'accelerator',
 9: 'accelerator…',
 10: 'accentuated',
 11: 'accept',
 12: 'acceptable',
 13: 'accepted',
 14: 'accerciser',
 15: 'access',
 16: 'accessed',
 17: 'accessibility',
 18: 'accessible',
 19: 'accessibles',
 20: 'according',
 21: 'accordion',
 22: 'ace',
 23: 'aces',
 24: 'across',
 25: 'act',
 26: 'acti',
 27: 'action',
 28: 'actionable',
 29: 'actions',
 30: 'activatable',
 31: 'activate',
 32: 'activated',
 33: 'active',
 34: 'actual',
 35: 'adaptive',
 36: 'add',
 37: 'added',
 38: 'addition',
 39: 'additional',
 40: 'addrcheck',
 41: 'address',
 42: 'addressbook',
 43: 'addresses',
 44: 'adds',
 45: 'add…',
 46: 'advance',
 47: 'advanced',
 48: 'advised',
 49: 'affects',
 50: 'again',
 51: 'agnes',
 52: 'aim',
 53: 'aisleriot',
 54: 'al',
 55: 'albert',
 56: 'alert',
 57: 'algorithm',
 58: 'alidate',
 59: 'all',
 60: 'alleys',
 61: 'allocate',
 62: 'allocated',
 63: 'allow',
 64:

In [ ]:
lines.head(10)

,hindi,english,length_eng,length_hin
0,START_ अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें _END,give application accessibility workout,4,10
1,START_ एक्सेर्साइसर पहुंचनीयता अन्वेषक _END,accerciser accessibility explorer,3,5
2,START_ निचले पटल के लिए डिफोल्ट प्लगइन खाका _END,default plugin layout bottom panel,5,9
3,START_ ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका _END,default plugin layout top panel,5,9
4,START_ उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है _END,list plugins disabled default,4,14
5,START_ अवधि को हाइलाइट रकें _END,highlight duration,2,6
6,START_ पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से की अवधि _END,duration highlight box selecting accessible nodes,6,12
7,START_ सीमांत बोर्डर के रंग को हाइलाइट करें _END,highlight border color,3,9
8,START_ हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। _END,color opacity highlight border,4,10
9,START_ भराई के रंग को हाइलाइट करें _END,highlight fill color,3,8


In [ ]:
from sklearn.model_selection import train_test_split
X, y = lines['english'], lines['hindi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((23799,), (5950,))

In [ ]:
X_train

11625    add remote         
5569     base card          
17025    step               
27010    push button        
18313    toolbar            
          ...               
21749    shortcut           
5432                        
864      lo gin helper      
15948    new library…       
23836    plugin dependencies
Name: english, Length: 23799, dtype: object

In [ ]:
encoder_input_data = np.zeros((2, max_length_src),dtype='float32')
decoder_input_data = np.zeros((2, max_length_tar),dtype='float32')
decoder_target_data = np.zeros((2, max_length_tar, num_decoder_tokens),dtype='float32')
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)
latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens+1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 2

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    786300      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    1022700     ['input_2[0][0]']                
                                                                                              

In [ ]:
model.save('eng-to-hindi.h5')

In [ ]:
a, b = next(generate_batch())

In [ ]:
a

[array([[  36., 1873.,    0., ...,    0.,    0.,    0.],
        [ 196.,  305.,    0., ...,    0.,    0.,    0.],
        [2190.,    0.,    0., ...,    0.,    0.,    0.],
        ...,
        [1587.,  615.,  156., ...,    0.,    0.,    0.],
        [2225.,  265., 1893., ...,    0.,    0.,    0.],
        [1261., 1197., 1740., ...,    0.,    0.,    0.]], dtype=float32),
 array([[1.000e+00, 1.440e+03, 2.621e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 6.640e+02, 1.863e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 1.217e+03, 5.740e+02, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        ...,
        [1.000e+00, 8.780e+02, 1.889e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 3.169e+03, 9.460e+02, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 1.769e+03, 5.030e+02, ..., 0.000e+00, 0.000e+00,
         0.000e+00]], dtype=float32)]

In [ ]:
b

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
X_train[4]

'list plugins disabled default'

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=10,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


371/371 [==============================] - 262s 680ms/step - loss: 1.1393 - accuracy: 0.2546 - val_loss: 0.9347 - val_accuracy: 0.3101
Epoch 2/10
371/371 [==============================] - 247s 666ms/step - loss: 0.7923 - accuracy: 0.3923 - val_loss: 0.6419 - val_accuracy: 0.4771
Epoch 3/10
371/371 [==============================] - 245s 659ms/step - loss: 0.4980 - accuracy: 0.5825 - val_loss: 0.4052 - val_accuracy: 0.6681
Epoch 4/10
371/371 [==============================] - 245s 658ms/step - loss: 0.2951 - accuracy: 0.7535 - val_loss: 0.2769 - val_accuracy: 0.7909
Epoch 5/10
371/371 [==============================] - 243s 654ms/step - loss: 0.1884 - accuracy: 0.8438 - val_loss: 0.2158 - val_accuracy: 0.8439
Epoch 6/10
371/371 [==============================] - 246s 661ms/step - loss: 0.1328 - accuracy: 0.8861 - val_loss: 0.1846 - val_accuracy: 0.8694
Epoch 7/10
371/371 [==============================] - 245s 660ms/step - loss: 0.1002 - accuracy: 0.9083 - val_loss: 0.1686 - val_accura

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 22ms/step
Input English sentence: add remote
Actual Hindi Translation:  जोड़ें रिमोट 
Predicted Hindi Translation:  जोड़ें रिमोट 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 24ms/step
Input English sentence: base card
Actual Hindi Translation:  आधार पत्ताः 
Predicted Hindi Translation:  आधार पत्ताः 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 21ms/step
Input English sentence: step
Actual Hindi Translation:  चरण अन्दर 
Predicted Hindi Translation:  चरण बाहर 


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
ref = [
    'this is moonlight'.split(),
    'Look, this is moonlight'.split(),
    'moonlight it is'.split()
]
test = 'it is moonlight'.split()
print('BLEU score for test-> {}'.format(sentence_bleu(ref, test)))
 
test01 = 'it is cat and moonlight'.split()
print('BLEU score for test01-> {}'.format(sentence_bleu(ref, test01)))

BLEU score for test-> 1.491668146240062e-154
BLEU score for test01-> 9.283142785759642e-155


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
